In [ ]:
# Importação de bibliotecas
import datetime
import ipywidgets as widgets
import json
import os
import pandas as pd

from IPython.display import display, clear_output
from requests_oauthlib import OAuth2Session

In [ ]:
# Constantes e Métodos
epoch_datetime = datetime.datetime(2025, 1, 1, 0, 0)

EPOCH_TIMESTAMP = epoch_datetime.timestamp()

QUANTITY_PAGES = 100

TRANSLATE_ACTIVITIES = {
    "Hike": "Trilha",
    "Crossfit": "Crossfit",
    "Walk": "Caminhada",
    "WeightTraining": "Levantamento de Peso",
    "Run": "Corrida",
    "Workout": "Treino",
    "Swim": "Natação",
    "Yoga": "Yoga",
    "MountainBikeRide": "Bicicleta",
}


def login_strava():
    # Baseado na documentação presente em https://developers.strava.com/
    # Defina local id, secret, e redirect_url - para gerar esses dados é necessário criar uma app no Strava
    CLIENT_ID = os.environ.get("STRAVA_CLIENT_ID")
    CLIENT_SECRET = os.environ.get("STRAVA_CLIENT_SECRET")
    REDIRECT_URL = "https://developers.strava.com"

    # Cria sessão
    session = OAuth2Session(client_id=CLIENT_ID, redirect_uri=REDIRECT_URL)

    # Define base auth url e escopo
    AUTH_BASE_URL = "https://www.strava.com/oauth/authorize"
    session.scope = ["activity:read"]

    # Gerar link de autorização
    auth_link = session.authorization_url(AUTH_BASE_URL)
    print(f"Clique aqui: {auth_link[0]}")
    redirect_response = input(f"Cole a redirect url aqui e pressione a tecla ENTER: ")

    # Obter token de sessão
    TOKEN_URL = "https://www.strava.com/api/v3/oauth/token"
    session.fetch_token(
        token_url=TOKEN_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        authorization_response=redirect_response,
        include_client_id=True,
    )

    return session


def generate_df_analysis_by_df(df, show_total):
    activity_counts_dict = df["sport_type"].value_counts().to_dict()
    df_activity_counts = pd.DataFrame.from_dict(
        activity_counts_dict, orient="index", columns=["Count"]
    )

    elapsed_time_by_activity_dict = (
        df.groupby("sport_type")["elapsed_time"].sum().to_dict()
    )
    elapsed_time_dict = {
        index: str(datetime.timedelta(seconds=value))
        for index, value in elapsed_time_by_activity_dict.items()
    }
    df_elapsed_time = pd.DataFrame.from_dict(
        elapsed_time_dict, orient="index", columns=["Elapsed Time"]
    )

    distance_by_activity_dict = df.groupby("sport_type")["distance"].sum().to_dict()
    distance_dict = {
        index: value / 1000 for index, value in distance_by_activity_dict.items()
    }
    df_distance = pd.DataFrame.from_dict(
        distance_dict, orient="index", columns=["Distance (km)"]
    )

    total_elevation_gain_by_activity_dict = (
        df.groupby("sport_type")["total_elevation_gain"].sum().to_dict()
    )
    df_elevation_gain = pd.DataFrame.from_dict(
        total_elevation_gain_by_activity_dict,
        orient="index",
        columns=["Elevation Gain (m)"],
    )

    concat_df = pd.concat(
        [df_activity_counts, df_elapsed_time, df_distance, df_elevation_gain], axis=1
    )
    concat_df.rename(index=TRANSLATE_ACTIVITIES, inplace=True)

    concat_df["Elapsed Time"] = pd.to_timedelta(concat_df["Elapsed Time"])

    if show_total:
        concat_df.loc["Total"] = concat_df.sum(numeric_only=False)
        total_seconds = concat_df.loc["Total", "Elapsed Time"].total_seconds()
        concat_df.loc["Total", "Elapsed Time"] = pd.to_timedelta(
            total_seconds, unit="s"
        )

    return concat_df


def get_number_days_year():
    today = datetime.date.today()
    first_day_year = datetime.date(today.year, 1, 1)
    return (today - first_day_year).days + 1

In [ ]:
# Login
session = login_strava()

# Request das informações do atleta logado
response = session.get("https://www.strava.com/api/v3/athlete")

# Validação do login
print("-" * 100)
print(f"Status Code: {response.status_code}")
print(f"Status: {response.reason}")
print(f"Conteúdo: {json.dumps(response.json(), indent=2)}")

In [ ]:
# Obter todas as atividades
all_activities = []
page = 1

while True:
    response = session.get(
        f"https://www.strava.com/api/v3/athlete/activities?after={EPOCH_TIMESTAMP}&page={page}&per_page=50"
    )

    if response.status_code != 200 or not response.json():
        print(response.status_code, response.json(), response.reason)
        break

    print(f"Dados Página {page}")
    all_activities.extend(response.json())
    page += 1

In [ ]:
# Crie um dataframe com as informações
df = pd.DataFrame.from_dict(all_activities, orient="columns")

# Limpeza/ajuste das colunas
df.drop("type", axis=1, inplace=True)
df.drop("resource_state", axis=1, inplace=True)
df.drop("athlete", axis=1, inplace=True)
df.start_date_local = pd.to_datetime(df.start_date_local)
df.start_date = pd.to_datetime(df.start_date)

df.tail(10)

In [ ]:
# Exibe dropdown de filtro
def statistics_by_activity(df_filtered):
    generated_df = generate_df_analysis_by_df(df_filtered, False)
    display(generated_df)


def on_change(change):
    with output:
        clear_output(wait=True)
        if change["type"] == "change" and change["name"] == "value":
            df_filtered = df[df["sport_type"] == change["new"]]
            display(df_filtered)
            statistics_by_activity(df_filtered)


select = widgets.Select(
    options=df.sport_type.unique(),
    value=df.sport_type.unique()[0],
    rows=9,
    description="Selecione:",
)

output = widgets.Output()

on_change({"type": "change", "name": "value", "new": select.value})
select.observe(on_change, names="value")
display(select, output)

In [ ]:
# Informações Gerais Anual
print(f"Total de Atividades: {len(df)}")
print(f"Tempo Total: {int(df['elapsed_time'].sum() / 3600)}h")
print(f"Distância Total: {df['distance'].sum() / 1000:.1f}km")
print(f"Elevação Total: {df['total_elevation_gain'].sum()}m")
print(
    f"Dias de Atividade: {len(df['start_date_local'].dt.date.unique())}/{get_number_days_year()}"
)

unique_df = generate_df_analysis_by_df(df, True)
display(unique_df)

In [ ]:
# Informações Semanais
def print_infos(this_week_grouped):
    print("-" * 35)
    print(WEEK)
    print(this_week_grouped)
    print("-" * 35)
    print(f"Distance: {round(this_week_grouped.distance.sum() / 1000, 2) }km")
    print(
        f"Time: {datetime.timedelta(seconds=int(this_week_grouped.elapsed_time.sum()))}"
    )


def generate_this_week_df(WEEK):
    start_date = pd.to_datetime(WEEKS_DICT[WEEK]["start_date"], utc=True)
    end_date = pd.to_datetime(WEEKS_DICT[WEEK]["end_date"], utc=True)
    this_week_df = df[
        (df["start_date_local"] >= start_date) & (df["start_date_local"] <= end_date)
    ]
    return this_week_df

# def anual_mondays():
#     d = datetime.date(2025, 4, 14)
#     d += datetime.timedelta(days=(7 - d.weekday()) % 7)
#     mondays = []
#     while d.year == 2025:
#         mondays.append(d)
#         d += datetime.timedelta(days=7)
#     return mondays

#
# for monday in anual_mondays():
#     print(monday)


WEEKS_DICT = {
    "Week 1": {"start_date": "2025-04-14", "end_date": "2025-04-20"},
    "Week 2": {"start_date": "2025-04-21", "end_date": "2025-04-27"},
}

WEEK = "Week 2"

this_week_df = generate_this_week_df(WEEK)

display(this_week_df)

this_week_grouped = this_week_df.groupby("sport_type").agg(
    {"distance": "sum", "elapsed_time": "sum"}
)

print_infos(this_week_grouped)